## 1. 데이터 불러오기

In [2]:
# 모듈 불러오기
import pandas as pd
import numpy as np
import time
import re
import requests
import urllib.parse

from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [3]:
# 데이터 불러오기
drama = pd.read_excel('./justwatch_second_1090.xlsx', index_col=0)
drama.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021),https://www.imdb.com/title/tt10919420/
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020),https://www.imdb.com/title/tt12079212/
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020),https://www.imdb.com/title/tt11612120/


## 2. 테스트 코드 작성

In [122]:
# distributor 테스트 코드
url = 'https://www.imdb.com/title/tt17346230/' + 'companycredits'
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

distributor_text = ''; distributor_set = set(); distributor_list = ''; distributor_companies = ''

distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(1)')[0]
distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
for company in distributor_companies:
    distributor_set.add(company.text)

distributor_set

{'DH Media Works Lab'}

In [161]:
# production 테스트 코드
url = 'https://www.imdb.com/title/tt8052538/' + 'companycredits'
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

production_text = ''; production_set = set(); production_list = ''; production_companies = ''

production_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2)')[0]
production_text = production_list.find('span', string='Production Companies').get_text(strip=True)

if production_text == 'Production Companies':
    production_companies = production_list.find_all(class_="ipc-metadata-list-item__label--link")
    for company in production_companies:
        production_set.add(company.text)

production_set

{'Cenic Media', 'Zhejiang Talent Television & Film Company'}

## 3. 제작사, 배급사 크롤링

### (1) 제작사, 배급사 전체 크롤링

In [172]:
# 제작사, 배급사 크롤링
for idx, row in tqdm_notebook(drama.iloc[test].iterrows()):

    # 집합 만들기
    production_set = set(); distributor_set = set()
    production_text = '';  production_list = ''; production_companies = ''
    distributor_text = ''; distributor_list = ''; distributor_companies = ''
    
    # 요청 헤더를 설정
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    # url 오류 확인
    try:
        url = row['IMDb_URL'] + 'companycredits'
        response = requests.get(url, headers=headers)

        # 정상 연결 확인
        if response.status_code == 200:
    
            # html 파싱
            soup = BeautifulSoup(response.text, 'html.parser')
        
            # 제작사
            try:
                production_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2)')[0]
                production_text = production_list.find('span', string='Production Companies').get_text(strip=True)

                if production_text == 'Production Companies':
                    production_companies = production_list.find_all(class_="ipc-metadata-list-item__label--link")
                    for company in production_companies:
                        production_set.add(company.text)
                    drama.at[idx, 'production'] = list(production_set)

            except:
                drama.at[idx, 'production'] = np.NaN

            # 배급사

            # 두 번째 항목인지 확인
            try:
                distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3)')[0]
                distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)

                if distributor_text == 'Distributors':
                    distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                    for company in distributor_companies:
                        distributor_set.add(company.text)
                    drama.at[idx, 'distributor'] = list(distributor_set)

            except:

                # 첫 번째 항목인지 확인
                try:
                    distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2)')[0]
                    distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)

                    if distributor_text == 'Distributors':
                        distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                        for company in distributor_companies:
                            distributor_set.add(company.text)
                        drama.at[idx, 'distributor'] = list(distributor_set)
                    else:
                        distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4)')[0]
                        distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)
    
                        if distributor_text == 'Distributors':
                            distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                            for company in distributor_companies:
                                distributor_set.add(company.text)
                            drama.at[idx, 'distributor'] = list(distributor_set)
                
                # 세 번째 항목인지 확인
                except:
                    distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4)')[0]
                    distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)

                    if distributor_text == 'Distributors':
                        distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                        for company in distributor_companies:
                            distributor_set.add(company.text)
                        drama.at[idx, 'distributor'] = list(distributor_set)
                    else:
                        distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2)')[0]
                        distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)
    
                        if distributor_text == 'Distributors':
                            distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                            for company in distributor_companies:
                                distributor_set.add(company.text)
                            drama.at[idx, 'distributor'] = list(distributor_set)
        
        else:
            print("Failed to retrieve data from:", idx)

    except:
        drama.at[idx, 'production'] = np.NaN
        drama.at[idx, 'distributor'] = np.NaN

C:\Users\mycom\AppData\Local\Temp\ipykernel_6764\4281036616.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, row in tqdm_notebook(drama.iloc[test].iterrows()):


0it [00:00, ?it/s]

### (2) 배급사가 결측치인 것만 제작사 다시 크롤링

In [183]:
# 제작사 다시 크롤링
# 배급사가 결측치인 것은 제작사까지 크롤링되지 않는 오류 발생

for idx, row in tqdm_notebook(drama[drama['distributor'].isnull()].iterrows()):

    # 집합 만들기
    production_set = set(); distributor_set = set()
    production_text = '';  production_list = ''; production_companies = ''
    distributor_text = ''; distributor_list = ''; distributor_companies = ''
    
    # 요청 헤더를 설정
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    # url 오류 확인
    try:
        url = row['IMDb_URL'] + 'companycredits'
        response = requests.get(url, headers=headers)

        # 정상 연결 확인
        if response.status_code == 200:
    
            # html 파싱
            soup = BeautifulSoup(response.text, 'html.parser')
        
            # 제작사
            production_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2)')[0]
            production_text = production_list.find('span', string='Production Companies').get_text(strip=True)

            if production_text == 'Production Companies':
                production_companies = production_list.find_all(class_="ipc-metadata-list-item__label--link")
                for company in production_companies:
                    production_set.add(company.text)
                drama.at[idx, 'production'] = list(production_set)
        
        else:
            print("Failed to retrieve data from:", idx)

    except:
        drama.at[idx, 'production'] = np.NaN
        drama.at[idx, 'distributor'] = np.NaN

C:\Users\mycom\AppData\Local\Temp\ipykernel_6764\3774700209.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, row in tqdm_notebook(drama.iloc[test].iterrows()):


0it [00:00, ?it/s]

### (3) 제작사가 5개 이상인 것 다시 크롤링

In [305]:
for idx, row in tqdm_notebook(drama.iloc[production_idx].iterrows()):

    # 집합 만들기
    production_set = set(); distributor_set = set()
    production_text = '';  production_list = ''; production_companies = ''
    distributor_text = ''; distributor_list = ''; distributor_companies = ''

    page = row['IMDb_URL'] + 'companycredits'

    # 하위 홈페이지를 selenium으로 열기
    driver = webdriver.Chrome()
    driver.get(page)
    
    try:
        more_link = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'span.ipc-see-more.sc-f55e45dc-0.gbpBtg.single-page-see-more-button-production'))
    )
        more_link.click()
    
    except:
        pass

    time.sleep(5)
    
    # 현재 페이지 내의 정보를 저장
    raw = driver.page_source
    
    # BeautifulSoup을 이용하여 html 파싱
    soup = BeautifulSoup(raw, 'html.parser')
    
    # 제작사
    production_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2)')[0]
    production_companies = production_list.find_all(class_="ipc-metadata-list-item__label--link")
    
    for company in production_companies:
        production_set.add(company.text)
    drama.at[idx, 'production'] = list(production_set)

    driver.close()

C:\Users\mycom\AppData\Local\Temp\ipykernel_6764\451686637.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, row in tqdm_notebook(drama.iloc[production_idx].iterrows()):


0it [00:00, ?it/s]

### (4) 배급사가 5개 이상인 것 다시 크롤링

In [456]:
# 배급사 크롤링
for idx, row in tqdm_notebook(drama.loc[[39, 607]].iterrows()):

    # 집합 만들기
    production_set = set(); distributor_set = set()
    production_text = '';  production_list = ''; production_companies = ''
    distributor_text = ''; distributor_list = ''; distributor_companies = ''

    page = row['IMDb_URL'] + 'companycredits'
    
    # 하위 홈페이지를 selenium으로 열기
    driver = webdriver.Chrome()
    driver.get(page)

    # more 열기
    for x in range(7):
        try:
            more_link = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.sc-f65f65be-0.bBlII > ul > div > span.ipc-see-more'))
            )
            more_link.click()
        
        except:
            pass

    time.sleep(15)
    
    # 현재 페이지 내의 정보를 저장
    raw = driver.page_source
    
    # BeautifulSoup을 이용하여 html 파싱
    soup = BeautifulSoup(raw, 'html.parser')
    
    # 배급사

    # 두 번째 항목인지 확인
    try:
        distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3)')[0]
        distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)

        if distributor_text == 'Distributors':
            distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
            for company in distributor_companies:
                distributor_set.add(company.text)
            drama.at[idx, 'distributor'] = list(distributor_set)

    except:

        # 첫 번째 항목인지 확인
        try:
            distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2)')[0]
            distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)

            if distributor_text == 'Distributors':
                distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                for company in distributor_companies:
                    distributor_set.add(company.text)
                drama.at[idx, 'distributor'] = list(distributor_set)
            else:
                distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4)')[0]
                distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)

                if distributor_text == 'Distributors':
                    distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                    for company in distributor_companies:
                        distributor_set.add(company.text)
                    drama.at[idx, 'distributor'] = list(distributor_set)
        
        # 세 번째 항목인지 확인
        except:
            distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4)')[0]
            distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)

            if distributor_text == 'Distributors':
                distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                for company in distributor_companies:
                    distributor_set.add(company.text)
                drama.at[idx, 'distributor'] = list(distributor_set)
            else:
                distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2)')[0]
                distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)

                if distributor_text == 'Distributors':
                    distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                    for company in distributor_companies:
                        distributor_set.add(company.text)
                    drama.at[idx, 'distributor'] = list(distributor_set)

    driver.close()

C:\Users\mycom\AppData\Local\Temp\ipykernel_6764\1311672837.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, row in tqdm_notebook(drama.loc[[39, 607]].iterrows()):


0it [00:00, ?it/s]

### (5) 최종 크롤링

In [22]:
# 제작사, 배급사 크롤링
for idx, row in tqdm_notebook(drama.iterrows()):

    # 집합 만들기
    production_set = set(); distributor_set = set()
    production_text = '';  production_list = ''; production_companies = ''
    distributor_text = ''; distributor_list = ''; distributor_companies = ''

    # url 오류 확인
    try:
        page = row['IMDb_URL'] + 'companycredits'

        # 하위 홈페이지를 selenium으로 열기
        driver = webdriver.Chrome()
        driver.get(page)
        
        # more 열기
        for x in range(20):
            try:
                more_link = WebDriverWait(driver, 1).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.sc-f65f65be-0.bBlII > ul > div > span.ipc-see-more'))
                )
                more_link.click()
            
            except:
                pass
    
        time.sleep(8)
        
        # 현재 페이지 내의 정보를 저장
        raw = driver.page_source
        
        # BeautifulSoup을 이용하여 html 파싱
        soup = BeautifulSoup(raw, 'html.parser')
            
        
        # 제작사
        try:
            production_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2)')[0]
            production_text = production_list.find('span', string='Production Companies').get_text(strip=True)

            if production_text == 'Production Companies':
                production_companies = production_list.find_all(class_="ipc-metadata-list-item__label--link")
                for company in production_companies:
                    production_set.add(company.text)
                drama.at[idx, 'production'] = list(production_set)

        except:
            drama.at[idx, 'production'] = np.NaN

        # 배급사

        # 두 번째 항목인지 확인
        try:
            distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3)')[0]
            distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)

            if distributor_text == 'Distributors':
                distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                for company in distributor_companies:
                    distributor_set.add(company.text)
                drama.at[idx, 'distributor'] = list(distributor_set)

        except:

            # 첫 번째 항목인지 확인
            try:
                distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2)')[0]
                distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)

                if distributor_text == 'Distributors':
                    distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                    for company in distributor_companies:
                        distributor_set.add(company.text)
                    drama.at[idx, 'distributor'] = list(distributor_set)
                else:
                    distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4)')[0]
                    distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)

                    if distributor_text == 'Distributors':
                        distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                        for company in distributor_companies:
                            distributor_set.add(company.text)
                        drama.at[idx, 'distributor'] = list(distributor_set)
            
            # 세 번째 항목인지 확인
            except:
                distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4)')[0]
                distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)

                if distributor_text == 'Distributors':
                    distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                    for company in distributor_companies:
                        distributor_set.add(company.text)
                    drama.at[idx, 'distributor'] = list(distributor_set)
                else:
                    distributor_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2)')[0]
                    distributor_text = distributor_list.find('span', string='Distributors').get_text(strip=True)

                    if distributor_text == 'Distributors':
                        distributor_companies = distributor_list.find_all(class_="ipc-metadata-list-item__label--link")
                        for company in distributor_companies:
                            distributor_set.add(company.text)
                        drama.at[idx, 'distributor'] = list(distributor_set)

        driver.close()

    except:
        drama.at[idx, 'production'] = np.NaN
        drama.at[idx, 'distributor'] = np.NaN

C:\Users\mycom\AppData\Local\Temp\ipykernel_22472\2611456241.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, row in tqdm_notebook(drama.iterrows()):


0it [00:00, ?it/s]

In [34]:
# 제작사 다시 크롤링
# 배급사가 결측치인 것은 제작사까지 크롤링되지 않는 오류 발생

for idx, row in tqdm_notebook(drama[drama['distributor'].isnull()].iterrows()):

    # 집합 만들기
    production_set = set(); distributor_set = set()
    production_text = '';  production_list = ''; production_companies = ''
    distributor_text = ''; distributor_list = ''; distributor_companies = ''

    # url 오류 확인
    try:
        page = row['IMDb_URL'] + 'companycredits'

        # 하위 홈페이지를 selenium으로 열기
        driver = webdriver.Chrome()
        driver.get(page)
        
        # more 열기
        for x in range(20):
            try:
                more_link = WebDriverWait(driver, 1).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'div.sc-f65f65be-0.bBlII > ul > div > span.ipc-see-more'))
                )
                more_link.click()
            
            except:
                pass
    
        time.sleep(8)
        
        # 현재 페이지 내의 정보를 저장
        raw = driver.page_source
        
        # BeautifulSoup을 이용하여 html 파싱
        soup = BeautifulSoup(raw, 'html.parser')
            
        
        # 제작사
        try:
            production_list = soup.select('#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(2)')[0]
            production_text = production_list.find('span', string='Production Companies').get_text(strip=True)

            if production_text == 'Production Companies':
                production_companies = production_list.find_all(class_="ipc-metadata-list-item__label--link")
                for company in production_companies:
                    production_set.add(company.text)
                drama.at[idx, 'production'] = list(production_set)

        except:
            drama.at[idx, 'production'] = np.NaN

        driver.close()

    except:
        drama.at[idx, 'production'] = np.NaN
        drama.at[idx, 'distributor'] = np.NaN

C:\Users\mycom\AppData\Local\Temp\ipykernel_22472\3332029294.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, row in tqdm_notebook(drama[drama['distributor'].isnull()].iterrows()):


0it [00:00, ?it/s]

## 4. 크롤링 결과 확인 후 저장

In [36]:
# 데이터 프레임 살펴보기
drama.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1090 entries, 0 to 1089
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               1090 non-null   object
 1   original_title      855 non-null    object
 2   year                1090 non-null   int64 
 3   season_episode      1090 non-null   int64 
 4   runtime             1090 non-null   object
 5   genre               1090 non-null   object
 6   age_rating          803 non-null    object
 7   Production country  1089 non-null   object
 8   IMDb_title          1088 non-null   object
 9   IMDb_URL            1088 non-null   object
 10  production          931 non-null    object
 11  distributor         1054 non-null   object
dtypes: int64(2), object(10)
memory usage: 143.0+ KB


In [37]:
# 처음 10개 살펴보기
drama.head(10)

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,production,distributor
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021),https://www.imdb.com/title/tt10919420/,[Siren Pictures],[Netflix]
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,"[Vlaamse Radio en Televisie (VRT), Eyeworks Fi...","[Netflix, Special Broadcasting Service (SBS), ..."
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020),https://www.imdb.com/title/tt12079212/,"[Third Man Entertainment, Greener Grass Produc...",[Netflix]
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,"[Warner Bros. Television, DC Entertainment, Ne...",[Netflix]
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020),https://www.imdb.com/title/tt11612120/,"[Studio N, Studio Dragon, Netflix]",[Netflix]
5,외교관,The Diplomat,2023,2,48min,"드라마, 전쟁, 스릴러, 코미디",15,미국,The Diplomat (TV Series 2023),https://www.imdb.com/title/tt17491088/,[Netflix Studios],[Netflix]
6,굿 닥터,The Good Doctor,2017,7,43min,드라마,15,미국,The Good Doctor (TV Series 2017–2024),https://www.imdb.com/title/tt6470478/,"[EnterMedia Content, Shore Z Productions, Sony...","[Videoload, Sony Pictures Television Internati..."
7,워킹 데드,The Walking Dead,2010,11,46min,"드라마, SF, 액션, 공포, 스릴러",19,미국,The Walking Dead (TV Series 2010–2022),https://www.imdb.com/title/tt1520211/,"[AMC Studios, American Movie Classics (AMC), I...","[FX Network, Orange Cinéma Séries, CNBC-e, Tud..."
8,"비르기트: 왕국, 권력, 영광","Borgen - Riget, Magten og Æren",2022,4,58min,"전쟁, 드라마, Made in Europe",15,덴마크,Borgen (TV Series 2010–2022),https://www.imdb.com/title/tt1526318/,"[Nordisk Film & TV-Fond, Danmarks Radio (DR), ...","[Videoload, hr-fernsehen, Österreichischer Run..."
9,The Flash,NaN,2014,9,45min,"액션, 드라마, SF",15,미국,The Flash (TV Series 2014–2023),https://www.imdb.com/title/tt3107288/,"[WarnerMedia, Warner Bros. Television, Berlant...","[DiziMax, Videoload, e2, HBO Max, CNBC-e, ProS..."


In [38]:
# 마지막 10개 살펴보기
drama.tail(10)

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,production,distributor
1080,꽃 같은 내 인생,植劇場 - 花甲男孩轉大人,2017,2,1시간 29분,"드라마, 코미디",15,대만,A Boy Named Flora A (TV Mini Series 2017),https://www.imdb.com/title/tt6284014/,"[Q Place, Oxygen Film Studio]",[TTV]
1081,방콕 프렌즈,เพื่อนรักบัดดี้,2019,1,40min,"드라마, 코미디",15,태국,Bangkok Buddies (TV Series 2019),https://www.imdb.com/title/tt11513508/,NaN,"[GMM 25, Netflix]"
1082,100만 엔의 여인들,100万円の女たち,2017,1,30min,"스릴러, 범죄, 드라마, 로맨스",19,일본,1000000 yen no Onnatachi (TV Mini Series 2017),https://www.imdb.com/title/tt6686450/,"[TV Tokyo, imageField]","[TV Tokyo, Netflix]"
1083,킬러 코미디언,Hasmukh,2020,1,33min,"범죄, 드라마, 스릴러, 코미디",NaN,인도,Hasmukh (TV Mini Series 2020),https://www.imdb.com/title/tt9359294/,"[Applause Entertainment Ltd., Weirdass Comedy,...","[Netflix, Netflix India]"
1084,Pink Zone,NaN,2007,1,15min,"드라마, 로맨스, 코미디",NaN,콜롬비아,Zona rosa (TV Series 2007– ),https://www.imdb.com/title/tt1050232/,[Telecolombia],[Canal RCN]
1085,달러,Dollar,2019,1,41min,"액션, 드라마, 로맨스",15,레바논,Dollar (TV Series 2019),https://www.imdb.com/title/tt10687564/,[Cedars Art Production - Sabbah Brothers],[Netflix]
1086,러브 나우,真愛趁現在,2012,1,45min,"드라마, 코미디",15,대만,Love Now (TV Series 2012–2013),https://www.imdb.com/title/tt6273116/,NaN,NaN
1087,탕탕 라이브,糖糖Online,2019,1,24min,드라마,15+,대만,Tang tang online (TV Series 2019),https://www.imdb.com/title/tt11330500/,[Left Culture],[PTS]
1088,대송궁사,大宋宫词,2021,1,45min,드라마,15+,China,Palace of Devotion (TV Series 2021),https://www.imdb.com/title/tt10742204/,[Beijing iQIYI Technology],NaN
1089,"엘리트들, 못다 한 이야기: 나디아 구스만",Elite Histórias Breves: Nadia Guzmán,2021,1,11min,"드라마, 로맨스",NaN,스페인,Elite Short Stories: Nadia Guzmán (TV Mini Ser...,https://www.imdb.com/title/tt14671790/,"[Producciones Zeta, Zeta Ficción]",[Netflix]


In [39]:
# 제작사, 배급사가 같은 경우 확인
drama[drama['production'] == drama['distributor']]

# IMDb 상 문제 없음 (크롤링 오류 아님)

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,production,distributor
79,그 땅에는 신이 없다,Godless,2017,1,1시간 5분,"서부, 드라마, 액션",19,미국,Godless (TV Mini Series 2017),https://www.imdb.com/title/tt5516154/,[Netflix],[Netflix]
181,더 소사이어티,The Society,2019,1,56min,"드라마, SF, 스릴러",19,미국,The Society (TV Series 2019),https://www.imdb.com/title/tt8778064/,[Netflix],[Netflix]
385,식샤를 할까요,รักล้นพุง,2021,1,1시간 0분,"드라마, 코미디",15+,"태국, 미국",Let's Eat (TV Series 2021),https://www.imdb.com/title/tt15978704/,[True CJ Creations],[True CJ Creations]
452,영매탐정 조즈카 히스이,霊媒探偵・城塚翡翠,2022,1,45min,"드라마, 범죄, 스릴러",15,일본,Medium (TV Series 2022),https://www.imdb.com/title/tt23009644/,[NTV+],[NTV+]
470,우리가 사랑에 빠졌던 모든 시간,In Love All Over Again,2023,1,45min,"코미디, 드라마, 로맨스",NaN,스페인,In Love All Over Again (TV Series 2023),https://www.imdb.com/title/tt21206964/,[Netflix],[Netflix]
479,바바리안,Barbaren,2020,2,47min,"액션, 드라마",19,독일,Barbarians (TV Series 2020–2022),https://www.imdb.com/title/tt9184986/,[Netflix],[Netflix]
548,심야식당: 도쿄 스토리,Midnight Diner: Tokyo Stories,2016,2,32min,드라마,15,일본,Midnight Diner: Tokyo Stories (TV Mini Series ...,https://www.imdb.com/title/tt6150576/,[Netflix],[Netflix]
599,"지미: 대체로 바보, 어쩌면 천재",Jimmy～アホみたいなホンマの話～,2018,1,46min,"드라마, 코미디",NaN,일본,Jimmy: The True Story of a True Idiot (TV Seri...,https://www.imdb.com/title/tt6866914/,[Netflix],[Netflix]
701,"노 굿, 닉!",No Good Nick,2019,2,29min,"코미디, 가족, 드라마",12,미국,No Good Nick (TV Series 2019),https://www.imdb.com/title/tt9039142/,[Netflix],[Netflix]
737,컨트리 컴포트,Country Comfort,2021,1,25min,"코미디, 드라마, 가족, 음악",12,미국,Country Comfort (TV Series 2021),https://www.imdb.com/title/tt11717394/,[Netflix],[Netflix]


In [40]:
# 제작사 결측치인 행 살펴보기
drama[drama['production'].isnull()]

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,production,distributor
40,천계지백사전설,天乩之白蛇传说,2018,1,45min,드라마,15+,China,The Destiny of White Snake (TV Series 2018),https://www.imdb.com/title/tt11482120/,NaN,[OnDemandChina]
81,군검사 도베르만,NaN,2022,1,1시간 5분,"드라마, 액션",15,대한민국,Military Prosecutor Do Bae Man (TV Series 2022),https://www.imdb.com/title/tt17346230/,NaN,"[tvN, Viu, Netflix]"
87,이노센스 원죄변호사,イノセンス 冤罪弁護士,2019,1,45min,"드라마, 범죄",15+,일본,Inosenusu Enzai Bengoshi (TV Series 2019),https://www.imdb.com/title/tt9712506/,NaN,[Netflix]
94,페르소나,NaN,2019,2,48min,"드라마, 스릴러",19,대한민국,Persona (TV Mini Series 2019–2023),https://www.imdb.com/title/tt10027990/,NaN,[Netflix]
97,도깨비,쓸쓸하고 찬란하神-도깨비,2016,1,53min,"SF, 드라마, 판타지, 로맨스, 코미디",15,대한민국,Guardian: The Lonely and Great God (TV Series ...,https://www.imdb.com/title/tt5994364/,NaN,"[tvN, Viu, Netflix, Disney+]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1068,No Regrets In Life,愛情發生在三天後,2022,1,1시간 14분,"드라마, 로맨스",NaN,대만,No Regrets in Life (TV Series 2022,https://www.imdb.com/title/tt20859930/,NaN,[Netflix]
1070,초연나건소사 : 첫사랑의 멜로디,初恋那件小事,2019,1,45min,"코미디, 드라마, 로맨스",NaN,China,A Little Thing Called First Love (TV Series 2019),https://www.imdb.com/title/tt11152058/,NaN,"[NewTV, Netflix]"
1073,청춘은 강펀치,รักหมัดหนัก,2020,1,43min,"드라마, 로맨스, 스포츠",15,태국,Hook (TV Series 2020– ),https://www.imdb.com/title/tt12656690/,NaN,[Netflix]
1081,방콕 프렌즈,เพื่อนรักบัดดี้,2019,1,40min,"드라마, 코미디",15,태국,Bangkok Buddies (TV Series 2019),https://www.imdb.com/title/tt11513508/,NaN,"[GMM 25, Netflix]"


In [190]:
# 배급사가 결측치인 행 살펴보기
drama[drama['distributor'].isnull()]

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,production,distributor
123,Love Class,NaN,2022,2,24min,"로맨스, 드라마",15,대한민국,Love Class (TV Series 2022–2023),https://www.imdb.com/title/tt19800744/,[Viki],NaN
124,복면 D,覆面D,2022,1,50min,드라마,15,일본,The Dirty D (TV Series 2022,https://www.imdb.com/title/tt18815516/,[Plush Productions],NaN
151,주벽 50,酒癖50,2022,1,41min,드라마,NaN,일본,The 50 (TV Series 2024– ) - Episode list,https://www.imdb.com/title/tt31804242/episodes...,NaN,NaN
415,모래에도 꽃이 핀다,NaN,2023,1,1시간 4분,"코미디, 드라마, 스릴러, 로맨스",15,대한민국,Like Flowers in Sand (TV Series 2023–2024),https://www.imdb.com/title/tt26924748/,NaN,NaN
425,마이 런웨이,NaN,2016,1,24min,"코미디, 드라마, 판타지, 로맨스",12+,대한민국,My Runway (TV Series 2016),https://www.imdb.com/title/tt6968336/,NaN,NaN
505,어사소오작,御赐小仵作,2021,1,39min,"스릴러, 코미디, 드라마",15,China,Yu ci xiao wu zuo (TV Series 2021),https://www.imdb.com/title/tt14792342/,NaN,NaN
552,그녀의 식탁,植劇場：五味八珍的歲月,2017,1,1시간 25분,드라마,12,대만,Mân hàn dà cân - Wûwèi bâ zhên de suìyuè: What...,https://www.imdb.com/title/tt7073222/,[Q Place],NaN
576,Entrapped,NaN,2022,1,32min,"범죄, 드라마, 스릴러",NaN,아이슬란드,Entrapped (TV Series 2022–),https://www.imdb.com/title/tt21964626/,NaN,NaN
626,"겨우, 서른",三十而已,2020,1,45min,드라마,15,China,San shi er yi (TV Series 2020– ),https://www.imdb.com/title/tt11357970/,NaN,NaN
692,마르세유,Marseille,2016,2,42min,"드라마, 범죄, Made in Europe",19,프랑스,Marseille (2016),https://www.imdb.com/title/tt4523050/,"[Pathé, LGM Productions, Janine Films, TF1 Fil...",NaN


In [43]:
# production 5개 이상
production_idx = []
for idx, row in drama[drama['production'].notnull()].iterrows():
    tmp = len(row['production'])
    if tmp >= 5:
        production_idx.append(idx)
len(production_idx), production_idx

(72,
 [3,
  6,
  7,
  8,
  12,
  14,
  19,
  29,
  31,
  32,
  34,
  37,
  38,
  39,
  44,
  48,
  50,
  52,
  56,
  57,
  65,
  66,
  67,
  69,
  74,
  75,
  89,
  108,
  112,
  113,
  132,
  145,
  205,
  212,
  216,
  218,
  227,
  230,
  241,
  280,
  297,
  299,
  306,
  339,
  367,
  371,
  414,
  439,
  464,
  472,
  499,
  513,
  577,
  603,
  607,
  637,
  666,
  669,
  671,
  692,
  722,
  725,
  760,
  774,
  821,
  852,
  869,
  983,
  992,
  1010,
  1051,
  1058])

In [45]:
# production 5개 이상인 것 보기
drama.iloc[production_idx]

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,production,distributor
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,"[Warner Bros. Television, DC Entertainment, Ne...",[Netflix]
6,굿 닥터,The Good Doctor,2017,7,43min,드라마,15,미국,The Good Doctor (TV Series 2017–2024),https://www.imdb.com/title/tt6470478/,"[EnterMedia Content, Shore Z Productions, Sony...","[Videoload, Sony Pictures Television Internati..."
7,워킹 데드,The Walking Dead,2010,11,46min,"드라마, SF, 액션, 공포, 스릴러",19,미국,The Walking Dead (TV Series 2010–2022),https://www.imdb.com/title/tt1520211/,"[AMC Studios, American Movie Classics (AMC), I...","[FX Network, Orange Cinéma Séries, CNBC-e, Tud..."
8,"비르기트: 왕국, 권력, 영광","Borgen - Riget, Magten og Æren",2022,4,58min,"전쟁, 드라마, Made in Europe",15,덴마크,Borgen (TV Series 2010–2022),https://www.imdb.com/title/tt1526318/,"[Nordisk Film & TV-Fond, Danmarks Radio (DR), ...","[Videoload, hr-fernsehen, Österreichischer Run..."
12,Suits,NaN,2011,9,44min,"코미디, 드라마",15,미국,Suits (TV Series 2011–2019),https://www.imdb.com/title/tt1632701/,"[Untitled Korsh Company, Hypnotic, Universal C...","[DiziMax, Videoload, My Network TV, M-Net, RTL..."
...,...,...,...,...,...,...,...,...,...,...,...,...
983,프로이트의 살인 해석,Freud,2020,1,53min,"드라마, 스릴러, 범죄",19,"독일, 오스트리아",Freud (TV Series 2020),https://www.imdb.com/title/tt8667956/,"[Netflix, Österreichischer Rundfunk (ORF), Bav...","[Netflix, ZDF Enterprises, Pandastorm Pictures..."
992,땅의 상속자들,Heirs to the Land,2022,1,54min,드라마,NaN,스페인,Heirs to the Land (TV Series 2022),https://www.imdb.com/title/tt13308276/,"[Atresmedia Televisión, Generalitat de Catalun...",[Netflix]
1010,엠파이어 옥토버페스트,Oktoberfest: Beer & Blood,2020,1,48min,"드라마, 역사, 스릴러",NaN,"체코 공화국, 독일",Oktoberfest: Beer & Blood (TV Mini Series 2020),https://www.imdb.com/title/tt10244612/,"[Bayerischer Rundfunk (BR), Maya Production, M...","[Leonine Distribution, Netflix, ONE, Das Erste..."
1051,거짓말의 거짓말,Mytho,2019,2,46min,"코미디, 드라마",19,프랑스,Mythomaniac (TV Series 2019–2021),https://www.imdb.com/title/tt10677432/,"[Netflix, Agence Nationale de Gestion des Oeuv...","[Federation Entertainment, ARTE, Netflix, ARTE..."


In [48]:
# production 5개
production_5 = []
for idx, row in drama[drama['production'].notnull()].iterrows():
    tmp = len(row['production'])
    if tmp == 5:
        production_5.append(idx)
len(production_5), production_5

(31,
 [3,
  12,
  14,
  32,
  34,
  44,
  50,
  52,
  56,
  65,
  66,
  69,
  89,
  112,
  113,
  218,
  230,
  280,
  299,
  339,
  367,
  439,
  577,
  603,
  637,
  671,
  725,
  774,
  852,
  869,
  983])

In [49]:
# production 5개
drama.iloc[production_5]

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,production,distributor
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,"[Warner Bros. Television, DC Entertainment, Ne...",[Netflix]
12,Suits,NaN,2011,9,44min,"코미디, 드라마",15,미국,Suits (TV Series 2011–2019),https://www.imdb.com/title/tt1632701/,"[Untitled Korsh Company, Hypnotic, Universal C...","[DiziMax, Videoload, My Network TV, M-Net, RTL..."
14,아웃랜더,Outlander,2014,7,1시간 0분,"드라마, 판타지, 로맨스, SF",NaN,미국,Outlander (TV Series 2014),https://www.imdb.com/title/tt3006802/,"[Sony Pictures Television, Soundtrack New York...","[Videoload, ORF 1, SRF zwei, Österreichischer ..."
32,위쳐,The Witcher,2019,4,57min,"액션, SF, 드라마, 판타지",19,미국,The Witcher (TV Series 2019),https://www.imdb.com/title/tt5180504/,"[Cinesite, Hivemind, Platige Image, Netflix, P...",[Netflix]
34,오자크,Ozark,2017,4,1시간 0분,"범죄, 드라마, 스릴러",19,미국,Ozark (TV Series 2017–2022),https://www.imdb.com/title/tt5071412/,"[Media Rights Capital (MRC), Man, Woman & Chil...",[Netflix]
44,베터 콜 사울,Better Call Saul,2015,6,50min,"범죄, 드라마",19,미국,Better Call Saul (TV Series 2015–2022),https://www.imdb.com/title/tt3032476/,"[Gran Via Productions, American Movie Classics...","[Amazon Instant Video, Videoload, Netflix Worl..."
50,나이트 에이전트,The Night Agent,2023,2,49min,"스릴러, 액션, 드라마",19,미국,The Night Agent (TV Series 2023,https://www.imdb.com/title/tt13918776/,"[Project X Entertainment, Sony Pictures Televi...",[Netflix]
52,루시퍼,Lucifer,2016,6,47min,"범죄, SF, 드라마, 판타지",19,미국,Lucifer (TV Series 2016–2021),https://www.imdb.com/title/tt4052886/,"[Warner Bros. Television, DC Entertainment, Ve...","[DiziMax, Videoload, Sky Replay, Sixx, FOXlife..."
56,덱스터,Dexter,2006,8,53min,"범죄, 드라마, 스릴러",19,미국,Dexter (TV Series 2006–2013),https://www.imdb.com/title/tt0773262/,"[The Colleton Company, Showtime Networks, John...","[ORF 1, 13th Street, Sky Replay, SRF zwei, e2,..."
65,엄브렐러 아카데미,The Umbrella Academy,2019,4,51min,"드라마, 판타지, SF, 액션, 코미디",15,미국,The Umbrella Academy (TV Series 2019–2024),https://www.imdb.com/title/tt1312171/,"[Borderline Entertainment, Netflix, Universal ...","[Universal Pictures Home Entertainment (UPHE),..."


In [44]:
# distributor 5개 이상
distributor_idx = []
for idx, row in drama[drama['distributor'].notnull()].iterrows():
    tmp = len(row['distributor'])
    if tmp >= 5:
        distributor_idx.append(idx)
len(distributor_idx), distributor_idx

(88,
 [1,
  6,
  7,
  8,
  9,
  12,
  14,
  17,
  19,
  22,
  24,
  26,
  27,
  28,
  29,
  36,
  37,
  39,
  42,
  44,
  46,
  48,
  51,
  52,
  55,
  56,
  58,
  60,
  61,
  62,
  63,
  68,
  71,
  72,
  74,
  86,
  99,
  103,
  112,
  114,
  115,
  122,
  132,
  145,
  149,
  156,
  167,
  171,
  172,
  173,
  177,
  195,
  198,
  209,
  226,
  239,
  241,
  270,
  276,
  278,
  312,
  318,
  331,
  332,
  333,
  356,
  357,
  368,
  377,
  431,
  467,
  486,
  499,
  516,
  561,
  603,
  607,
  610,
  637,
  728,
  867,
  872,
  886,
  983,
  996,
  1000,
  1010,
  1049])

In [42]:
# distributor 5개 이상인 것 보기
drama.iloc[distributor_idx]

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,production,distributor
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,"[Vlaamse Radio en Televisie (VRT), Eyeworks Fi...","[Netflix, Special Broadcasting Service (SBS), ..."
6,굿 닥터,The Good Doctor,2017,7,43min,드라마,15,미국,The Good Doctor (TV Series 2017–2024),https://www.imdb.com/title/tt6470478/,"[EnterMedia Content, Shore Z Productions, Sony...","[Videoload, Sony Pictures Television Internati..."
7,워킹 데드,The Walking Dead,2010,11,46min,"드라마, SF, 액션, 공포, 스릴러",19,미국,The Walking Dead (TV Series 2010–2022),https://www.imdb.com/title/tt1520211/,"[AMC Studios, American Movie Classics (AMC), I...","[FX Network, Orange Cinéma Séries, CNBC-e, Tud..."
8,"비르기트: 왕국, 권력, 영광","Borgen - Riget, Magten og Æren",2022,4,58min,"전쟁, 드라마, Made in Europe",15,덴마크,Borgen (TV Series 2010–2022),https://www.imdb.com/title/tt1526318/,"[Nordisk Film & TV-Fond, Danmarks Radio (DR), ...","[Videoload, hr-fernsehen, Österreichischer Run..."
9,The Flash,NaN,2014,9,45min,"액션, 드라마, SF",15,미국,The Flash (TV Series 2014–2023),https://www.imdb.com/title/tt3107288/,"[WarnerMedia, Warner Bros. Television, Berlant...","[DiziMax, Videoload, e2, HBO Max, CNBC-e, ProS..."
...,...,...,...,...,...,...,...,...,...,...,...,...
983,프로이트의 살인 해석,Freud,2020,1,53min,"드라마, 스릴러, 범죄",19,"독일, 오스트리아",Freud (TV Series 2020),https://www.imdb.com/title/tt8667956/,"[Netflix, Österreichischer Rundfunk (ORF), Bav...","[Netflix, ZDF Enterprises, Pandastorm Pictures..."
996,블렛츨리 서클: 샌프란시스코,The Bletchley Circle: San Francisco,2018,1,45min,"범죄, 드라마, 스릴러, 역사",15,영국,The Bletchley Circle: San Francisco (TV Series...,https://www.imdb.com/title/tt7978912/,"[Omnifilm Entertainment, World Productions]","[Netflix, Kew Media Group, Citytv, BritBox, Yl..."
1000,고려거란전쟁,NaN,2023,1,53min,"드라마, 전쟁, 액션, 역사",15,대한민국,Korea-Khitan War (TV Series 2023– ),https://www.imdb.com/title/tt26767539/,"[Vive Studios, KBS, Monster Union]","[KOCOWA, Netflix, Rakuten Viki, Wavve, OnDeman..."
1010,엠파이어 옥토버페스트,Oktoberfest: Beer & Blood,2020,1,48min,"드라마, 역사, 스릴러",NaN,"체코 공화국, 독일",Oktoberfest: Beer & Blood (TV Mini Series 2020),https://www.imdb.com/title/tt10244612/,"[Bayerischer Rundfunk (BR), Maya Production, M...","[Leonine Distribution, Netflix, ONE, Das Erste..."


In [46]:
# distributor 5개
distributor_5 = []
for idx, row in drama[drama['distributor'].notnull()].iterrows():
    tmp = len(row['distributor'])
    if tmp == 5:
        distributor_5.append(idx)
len(distributor_5), distributor_5

(18,
 [1,
  112,
  132,
  177,
  226,
  318,
  331,
  356,
  431,
  467,
  499,
  516,
  561,
  603,
  610,
  867,
  872,
  1010])

In [47]:
# distributor 5개인 것 살펴보기
drama.iloc[distributor_5]

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,production,distributor
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,"[Vlaamse Radio en Televisie (VRT), Eyeworks Fi...","[Netflix, Special Broadcasting Service (SBS), ..."
112,리버데일,Riverdale,2017,7,42min,"로맨스, 범죄, 드라마, 스릴러",15,미국,Riverdale (TV Series 2017–2023),https://www.imdb.com/title/tt5420376/,"[Warner Bros. Television, Archie Comics Public...","[DiziMax, Warner Archive Collection, Netflix, ..."
132,챔피언의 비트,Champion,2023,1,45min,드라마,NaN,"미국, 영국",Champion (TV Series 2023),https://www.imdb.com/title/tt21364468/,"[New Pictures, Netflix, Balloon Entertainment,...","[BBC iPlayer, Netflix, BBC Three, BBC One, Bri..."
177,호텔 델루나,NaN,2019,1,1시간 20분,"드라마, 코미디, SF, 판타지, 공포, 로맨스, 액션",15,대한민국,Hotel Del Luna (TV Series 2019),https://www.imdb.com/title/tt10220588/,"[Studio Dragon, GTist]","[Tving, Netflix, Amazon Prime Video, tvN, Disn..."
226,데드윈드,Karppi,2018,3,46min,"드라마, 범죄, 스릴러",NaN,핀란드,Deadwind (TV Series 2018–2021),https://www.imdb.com/title/tt6616260/,"[H & V Entertainment, Dionysos Films, Odeon Fi...","[About Premium Content, Netflix, Fernsehjuwele..."
318,Red Rose,NaN,2022,1,44min,"스릴러, 드라마, 공포",15,영국,Red Rose (TV Series 2022),https://www.imdb.com/title/tt19788500/,"[Eleven, British Broadcasting Corporation (BBC...","[Netflix, BBC iPlayer, BBC Three, BBC One, Bri..."
331,푸른 바다의 전설,NaN,2016,2,1시간 1분,"코미디, 드라마, SF, 판타지, 로맨스",15,대한민국,Legend of the Blue Sea (TV Series 2016–2017),https://www.imdb.com/title/tt5766194/,NaN,"[Studio Dragon, Watcha, Netflix, Seoul Broadca..."
356,별에서 온 그대,NaN,2013,1,55min,"코미디, 드라마, SF, 판타지, 로맨스",15,대한민국,My Love from Another Star (TV Series 2013–2014),https://www.imdb.com/title/tt3469052/,"[Seoul Broadcasting System (SBS), HB Entertain...","[Watcha, Netflix, Seoul Broadcasting System (S..."
431,수터블 보이,A Suitable Boy,2020,2,58min,"드라마, 로맨스",15,"영국, 인도",A Suitable Boy (TV Series 2020),https://www.imdb.com/title/tt10795574/,"[Production Scope, Lookout Point]","[Netflix, Acorn TV, BBC One, British Broadcast..."
467,블랙 라이트닝,Black Lightning,2018,4,42min,"드라마, 액션, SF",15,미국,Black Lightning (TV Series 2018–2021),https://www.imdb.com/title/tt6045840/,"[Warner Bros. Television, Berlanti Productions...","[DiziMax, Netflix, HBO Max, Warner Home Video,..."


In [50]:
# 데이터 프레임 저장
drama.to_excel('./imdb_prod_dist_수정본.xlsx')

## 5. EDA

In [75]:
# 제작사 개수 세기
drama['len_prod'] = 0
tmp = drama.loc[drama['production'].notnull(), 'production'].apply(lambda x: len(x[0]))
drama.loc[drama['production'].notnull(), 'len_prod'] = tmp

In [83]:
drama.loc[1, 'production']

"['Vlaamse Radio en Televisie (VRT)', 'Eyeworks Film & TV Drama']"

In [77]:
drama.loc[drama['production'].notnull(), 'production'].apply(lambda x: len(x[0]))

0       1
1       1
2       1
3       1
4       1
       ..
1084    1
1085    1
1087    1
1088    1
1089    1
Name: production, Length: 931, dtype: int64

In [73]:
for idx, row in drama[drama['production'].notnull()].iterrows():
    drama.loc[idx, 'len_prod'] = len(row['production'])

In [76]:
drama.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,production,distributor,len_prod
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021),https://www.imdb.com/title/tt10919420/,['Siren Pictures'],['Netflix'],1
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,"['Vlaamse Radio en Televisie (VRT)', 'Eyeworks...","['Netflix', 'Special Broadcasting Service (SBS...",1
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020),https://www.imdb.com/title/tt12079212/,"['Third Man Entertainment', 'Greener Grass Pro...",['Netflix'],1
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,"['Warner Bros. Television', 'DC Entertainment'...",['Netflix'],1
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020),https://www.imdb.com/title/tt11612120/,"['Studio N', 'Studio Dragon', 'Netflix']",['Netflix'],1


In [ ]:
drama